In [51]:
from jac import calc_jacobian
from transformer_lens import HookedTransformer
from transformers import AutoTokenizer
import torch as t
import einops
import matplotlib.pyplot as plt
from datasets import load_dataset
device = t.device("cuda") if t.cuda.is_available() else t.device("cpu")


model = HookedTransformer.from_pretrained("gpt2-small")
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")

ds = load_dataset("wikimedia/wikipedia", "20231101.en")
ds.set_format(type="torch", cotorch.Size([1, 181, 16, 768])lumns=["text"])

# Batch some inputs
batch_size = 2
context_size = 1024


Loaded pretrained model gpt2-small into HookedTransformer


Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/41 [00:00<?, ?it/s]

In [52]:
num_dims = 32
block = 1

for i, batch in zip(range(2), ds['train'].iter(batch_size=batch_size)):
    flattened = "".join([char for text in batch["text"] for char in text])
    #for j in range(len(flattened)//context_size):
    for j in range(1):
        tokens = tokenizer(
                    flattened[j * context_size : (j + 1) * context_size],
                    max_length=1024,
                    return_tensors="pt").input_ids

        jac = calc_jacobian(model.blocks[0].hook_resid_mid, model.blocks[0].hook_resid_post, model, tokens,
                            stop_idx=num_dims)
        print(jac.shape)
        plt.figure(figsize=(20, 20))
        seq_len = 100
        f = plt.imshow(einops.rearrange(jac[0, :seq_len, :, :num_dims] - t.eye(num_dims, device=device),
                                      "(s1 s2) d_down d_up -> (s1 d_down) (s2 d_up)",
                                      s1=int(seq_len**0.5)).cpu())
        plt.colorbar(f)
        plt.show()



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [48]:
model

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0-11): 12 x TransformerBlock(
      (ln1): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (h